In [2]:
import cv2
import numpy as np

In [3]:
#import haarcascades
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')

In [4]:
#detect face and eye on webcam + HoughCircle
video_capture=cv2.VideoCapture(0)
while True:
    ret, frame= video_capture.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blurred=cv2.blur(gray,(3,3))

    faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))
    eyes=eye_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))
    #HoughCircle works fine in proper lighting
    circles=cv2.HoughCircles(gray_blurred,cv2.HOUGH_GRADIENT,1,100,param1=50,param2=30,minRadius=10,maxRadius=30)
    if circles is not None:
        circles = np.round(circles[0, :]).astype(int)
    
    # Draw the detected circles on the original image
        for (x, y, r) in circles:
            cv2.circle(frame, (x, y), r, (0, 0, 255), 2)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
    for (x,y,w,h) in eyes:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),1)
        
    x,y,w,h=eyes[0,:]
    x1,y1,w1,h1=eyes[1,:]
    
#     w1=eyes[0,2]
#     h1=eyes[0,3]
    
    
    
    if (x<x1):
        left_eye_x=x
        left_eye_y=y
        left_eye_w=w
        left_eye_h=h
        right_eye_x=x1
        right_eye_y=y1
        right_eye_w=w1
        right_eye_h=h1
    else:
        left_eye_x=x1
        left_eye_y=y1
        left_eye_w=w1
        left_eye_h=h1
        right_eye_x=x
        right_eye_y=y
        right_eye_w=w
        right_eye_h=h
        
    print(left_eye_x,left_eye_y, right_eye_x, right_eye_y)
    
    cv2.imshow('Detection',frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
video_capture.release()  
cv2.destroyAllWindows()

267 203 361 188
265 212 358 200
265 214 356 200
264 215 354 200
263 214 353 197
263 213 355 199
263 214 352 199
262 215 351 199


In [5]:
print(x1,y1,w1,h1)

351 199 77 77


In [11]:
#only HoughCircle is working
video_capture=cv2.VideoCapture(0)
while True:
    ret, frame=video_capture.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_blurred=cv2.blur(gray,(3,3))
#     x,y,w,h=faces[0,:]
#     roi=gray_blurred[y:y+h,x:x+w]
    #HoughCircle works fine in proper lighting
    circles=cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,100,param1=50,param2=30,minRadius=10,maxRadius=30)
    
    print(circles)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype(int)
#     circles=circles[0]
        print(circles)
        for (x,y,r) in circles:
            cv2.circle(frame,(x,y),r,(0,255,0),2)
        
    cv2.imshow('Detection',frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
#         print(circles)
#         break
    x,y,r=circles[0,:]
    x1,y1,r1=circles[1,:]

    if (x<x1):
        left_pupil_x=x
        left_pupil_y=y
        right_pupil_x=x1
        right_pupil_y=y1
    else:
        left_pupil_x=x1
        left_pupil_y=y1
        right_pupil_x=x
        right_pupil_y=y
    

    # Display the image with the detected circles
#     cv2.imshow('Detection',frame)
#     if cv2.waitKey(1) & 0XFF == ord('q'):
#         break
video_capture.release()  
cv2.destroyAllWindows()
        

[[[363.5 424.5  11.8]
  [ 52.5 179.5  21.7]]]
[[364 424  12]
 [ 52 180  22]]


In [12]:
horizontalRatio_left=(left_pupil_x-left_eye_x)/(left_eye_w-10)
horizontalRatio_right=(right_pupil_x-right_eye_x)/(right_eye_w-10)  # w will almost be same for both eye
avg_horizontalRatio=(horizontalRatio_left+horizontalRatio_right)/2

verticalRatio_left=(left_pupil_y-left_eye_y)/(left_eye_h-10)
verticalRatio_right=(right_pupil_y-right_eye_y)/(right_eye_h-10)
avg_verticalRatio=(verticalRatio_left+verticalRatio_right)/2

In [13]:
def is_left_top():
    if(avg_horizontalRatio<0.35 & avg_verticalRatio<0.35):
        return True;

def is_left_middle():
    if(avg_horizontalRatio<0.35 & 0.35<avg_verticalRatio<0.65):
        return True;    
def is_left_bottom():
    if(avg_horizontalRatio<0.35 & avg_verticalRatio>0.65):
        return True;
    
def is_center_top():
    if(0.35<avg_horizontalRatio<0.65 & avg_verticalRatio<0.35):
        return True;

def is_center_middle():
    if(0.35<avg_horizontalRatio<0.65 & 0.35<avg_verticalRatio<0.65):
        return True;
    
def is_center_bottom():
    if(0.35<avg_horizontalRatio<0.65 & avg_verticalRatio>0.65):
        return True;
    
def is_right_top():
    if(avg_horizontalRatio>0.65 & avg_verticalRatio<0.35):
        return True;
    
def is_right_middle():
    if(avg_horizontalRatio>0.65 & 0.35<avg_verticalRatio<0.65):
        return True;
    
def is_right_bottom():
    if(avg_horizontalRatio>0.65 & avg_verticalRatio>0.65):
        return True;

In [18]:
#debugging code for pupil,face and eye

frame=cv2.imread("image.jpeg")
gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray_blurred=cv2.blur(gray,(3,3))

faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))
circles=cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,100,param1=50,param2=30,minRadius=10,maxRadius=30)
eyes=eye_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))

circles = np.round(circles[0, :]).astype(int)
for (x,y,r) in circles:
    cv2.circle(frame,(x,y),r,(0,255,0),2)
        
# for (x,y,w,h) in faces:
#     cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),1)

for (x,y,w,h) in eyes:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),1)
        
# print(circles)
# print(type(circles))
# # print(faces[0,1])
# x,y,w,h=faces[0,:]
# print(x,y)
    
cv2.imshow('Detection',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
circles

array([[358, 472,  19],
       [296, 384,  18]])

In [15]:
circles[1,:]

array([296, 384,  18])

In [48]:
x

231

In [64]:
eyes

()